In [1]:
import cv2

def count_frames(path):
    video = cv2.VideoCapture(path)
    total = video.get(cv2.CAP_PROP_FRAME_COUNT)
    return total
    

In [2]:
def sequence(number):
    number = str(number)
    if len(number) == 1:
        number = '00' + number
    elif len(number) == 2:
        number = '0' + number
    name = 'SEQ'+number
    return name

In [3]:
def get_seq_dir(number):
    base_dir = r'C:\Users\oussamaJB\Desktop\Full data\data'
    seq_name = sequence(number)
    return base_dir + "\\" + seq_name + '.avi'

In [4]:
import cv2
import numpy as np

# enhances the contrast of the image

def enhance(img):
    # converting to LAB color space
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)
    
    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # Stacking the original image with the enhanced image
    return enhanced_img

In [5]:
import numpy as np

def detect_circle(img, width, height):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # detect circles in the image
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
    # ensure at least some circles were found
    if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")
        valid_circles = [ tupl for tupl in circles if (tupl[0]>410 and tupl[0]<870 and tupl[1]>200 and tupl[1]<520 ) ]
        # sort circles
        # sorted_by_radius = sorted(circles, key=lambda tup: tup[2], reverse = True)
        
        # get the circle with the maximum radius 
        if len(valid_circles) > 0:
            max_circle = max(valid_circles,key=lambda item:item[2])
        else:
            return None
        # returning the coordinates of the center of the circle
        x, y, r = max_circle
        return (x,y)
    else: 
        return None

In [6]:
import os

def get_frames(path, count, seq_nb,total):
    cap = cv2.VideoCapture(path)
    
    good = 0
    good_frames = []
    points = []

    for index in range(total):
        
        ret, frame = cap.read()
        
        # uncomment the line below to adjust the contrast
        # frame=enhance(frame)

        if ret :
            circle_center = detect_circle(frame,100,100)
        else:
            circle_center = None
        if circle_center is not None :
            good +=1
            good_frames.append(index)
            points.append(circle_center)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    if good == 0:
        print('Could not locate the cell in sequence {}'.format(seq_nb))
        return
    elif good < count :
        print('Could not take enough shots in sequence {}'.format(seq_nb))
        return
    
    # Pick a step to iterate over the frames 
    step = max(int(good / count),1)
    frame_index = 1
    
    # Configure the save path
    save_dir = r'C:\Users\oussamaJB\Documents\PFA\no' 
    os.mkdir(save_dir+"\\"+sequence(seq_nb))
    save_dir = save_dir  + "\\" + sequence(seq_nb) + "\\"
    
    for index in range(0,good, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES,good_frames[index])
        ret, frame = cap.read()
        x, y = points[index]
        x = min(max(410,x),870)
        y = min(max(200,y),520)
        cropped_frame = frame[y-180:y+180,x-180:x+180]
        cv2.imwrite(save_dir+'frame{}.jpg'.format(good_frames[index]),cropped_frame)
        frame_index += 1
        if frame_index == count+1:
            break
        
    print('number of good frames in sequence {} : {}'.format(seq_nb,good))
    #print(good_frames)
    cv2.destroyAllWindows()

In [9]:
import pandas as pd
import os
from math import isnan

cor_sheet_path = r'C:\Users\oussamaJB\Desktop\Full data\Sheet de correspondance.csv'
annot_sheets_path = r'C:\Users\oussamaJB\Desktop\Full data\annots'

seq_df = pd.read_csv(cor_sheet_path)
count=0 
annotated=0

for seq in seq_df['original name']:
    seq_sheet = 'Time-lapse # '+seq[:seq.index('P')-1]+' summary.xlsx'
    if not os.path.exists(annot_sheets_path+'\\'+seq_sheet):
        #print(seq_sheet)
        #count += 1
        continue
    else:
        annot = pd.read_excel(annot_sheets_path+'\\'+seq_sheet,header=None,index_col=0)
        part = seq[seq.index('P')+1:seq.index('P')+3]
        if(part[0]=='0'): 
            part = int(part[1])
        else:
            part = int(part)
        part -=1
        pb = list(annot.loc['2PB'])
        pn1 = list(annot.loc['1PN'])
        pn2 = list(annot.loc['2PN'])
        pn3 = list(annot.loc['3PN'])
        pnf = list(annot.loc['PN Fading'])
        ras = list(annot.loc['ras'])
        if isnan(pb[part]) and isnan(pn1[part]) and isnan(pn2[part]) and isnan(pn3[part]) and isnan(ras[part]):
            continue
        else:
            count +=1
        

print('number of annotated videos is {}'.format(count))

number of annotated videos is 285


In [15]:
from math import ceil,isnan


# Put here the number of videos you want to process
NB_VIDS = 5

for i in range(1,NB_VIDS+1):
    #print(sequence(i))
    row = seq_df.iloc[i-1]
    seq_name = row[0]
    seq = row[1]
    print(seq)
    print(seq_name)
    seq_sheet = 'Time-lapse # '+seq[:seq.index('P')-1]+' summary.xlsx'
    #print(seq_sheet)
    if not os.path.exists(annot_sheets_path+'\\'+seq_sheet):
        continue
    else:
        annot = pd.read_excel(annot_sheets_path+'\\'+seq_sheet,header=None,index_col=0)
        part = seq[seq.index('P')+1:seq.index('P')+3]
        if(part[0]=='0'): 
            part = int(part[1])
        else:
            part = int(part)
        part -=1
        pb = list(annot.loc['2PB'])
        pn1 = list(annot.loc['1PN'])
        pn2 = list(annot.loc['2PN'])
        pn3 = list(annot.loc['3PN'])
        pnf = list(annot.loc['PN Fading'])
        ras = list(annot.loc['ras'])
        if isnan(pb[part]) and isnan(pn1[part]) and isnan(pn2[part]) and isnan(pn3[part]) and isnan(ras[part]):
            continue
        else:
            dic = {}
            if not isnan(ras[part]):
                dic = { '2PB':0 , '1PN':0 , '2PN':0 , '3PN':0 , 'PNf':0}
            else:
                dic = { '2PB':pb[part] , '1PN':pn1[part] , '2PN':pn2[part] , '3PN':pn3[part] , 'PNf':pnf[part]}
            df= pd.DataFrame.from_dict(dic,orient='index')
            frame_limit=400
            if not isnan(pnf[part]):
                frame_limit = ceil(pnf[part]*12)
            elif not isnan(pn3[part]):
                frame_limit = ceil(pn3[part]*12)
            elif not isnan(pn2[part]):
                frame_limit = ceil(pn2[part]*12)
            elif not isnan(pn1[part]):
                frame_limit = ceil(pn1[part]*12)
            elif not isnan(pb[part]):
                frame_limit = ceil(pb[part]*12)
            data_dir = r'C:\Users\oussamaJB\Desktop\Full data\data' + "\\" + seq_name +'.avi'
            get_frames(data_dir,20,i,frame_limit)
            #print(df.index)
            df.to_csv('./no/'+seq_name+'.csv')


MTL-0130-0EFD-D6E0-[37]-P01-FP3.avi
SEQ001
number of good frames in sequence 1 : 250
MTL-0130-0EFD-D6E0-[37]-P02-FP3.avi
SEQ002
number of good frames in sequence 2 : 260
MTL-0130-0EFD-D6E0-[37]-P03-FP3.mp4
SEQ003
number of good frames in sequence 3 : 253
MTL-0130-0EFD-D6E0-[37]-P04-FP3.mp4
SEQ004
number of good frames in sequence 4 : 283
